In [1]:
import warnings
warnings.filterwarnings('ignore')

import matplotlib
matplotlib.use('Agg')

from hparams.normal_hparams import create_hparams
from model import LightningModel

In [3]:
stt_hparams, tts_hparams = create_hparams()
model = LightningModel(stt_hparams, tts_hparams)
print('done')

done


In [5]:
from datasets import LJDataModule
data_module = LJDataModule(tts_hparams)
print('done')

done


In [12]:
import matplotlib.pyplot as plt
import librosa.display
from pytorch_lightning.callbacks import Callback

class PrintingCallback(Callback):
    def on_batch_end(self, trainer, pl_module):
        if pl_module.model.global_step % 10 == 0:
            _mel = pl_module.tts_outputs['mels'].data.cpu().numpy() 
            _pred = pl_module.tts_outputs['pred'].data.cpu().numpy()
            
            fig = plt.figure(figsize=[18, 6])
            plt.subplot(2, 1, 1)
            plt.title('Ground Truth')
            librosa.display.specshow(_mel[0])
            plt.subplot(2, 1, 2)
            plt.title('Predicted')
            librosa.display.specshow(_pred[0])
            fig.savefig('logs/fig.png')
                
callbacks = [PrintingCallback()]

In [13]:
import pytorch_lightning as pl

trainer = pl.Trainer(gpus=1, callbacks=callbacks)
trainer.fit(model, data_module)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores

  | Name  | Type  | Params
--------------------------------
0 | model | Model | 24.3 M
--------------------------------
24.3 M    Trainable params
1         Non-trainable params
24.3 M    Total params
97.132    Total estimated model params size (MB)


1